# MATH 2071 Lab 7 - Matrix factorization

Matthew Ragoza

2022-02-16

In [ ]:
%matplotlib inline
import sys
import numpy as np
from numpy.linalg import norm
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

sys.path.append('../lab-5/code')
import gallery

def print_mat(round=4, **kwargs):
    for k, v in kwargs.items():
        print(f'{k} =\n{np.round(v, round)}')

## Exercise 1 - Gram Schmidt method

The purpose of this exercise is to implement the Gram Schmidt method for finding a matrix whose columns are an orthonormal set of vectors that span the same space as the columns of an input matrix.

I implemented a function `gs(X)` that takes a matrix as input and produces an output matrix whose columns are linearly independent, normalized to unit length, and span the same space is the input matrix.

In [2]:
def gs(X):
    '''
    Gram Schmidt method.

    Args:
        X: M x N matrix.
    Returns:
        M x N_Q matrix with orthonormal columns
            that span the same space as X.
    '''
    m, n = X.shape
    
    # accrue orthonormal vectors in Q
    Q = []
    
    # for each column,
    for j in range(n):
        
        X_j = X[:,j]
    
        # for each previous orthogonal column,
        for i in range(len(Q)):

            Q_i = Q[i]
    
            # compute the dot product
            r_ij = Q_i @ X_j
            
            # subtract the projection
            #   this makes them orthogonal
            X_j = X_j - Q_i * r_ij

        # normalize the vector
        r_jj = norm(X_j)

        if r_jj > 0:
            Q.append(X_j / r_jj)
             
    return np.stack(Q, axis=1)

Next, I wrote the function `exercise1(X)` which takes a matrix $X$ as input and applies the Gram Schmidt process to obtain a matrix $Q$. Then it checks if the columns of $Q$ are orthogonal by computing the error norms $\|I - Q^\top Q\|$ and $\|I - QQ^\top\|$. In additon, the number of rows and columns in matrix $Q$ are returned from the function.

In [3]:
def exercise1(X):
    m, n = X.shape
    Q = gs(X)
    Im = np.eye(m)
    In = np.eye(n)
    e1 = norm(Im - Q @ Q.T)
    e2 = norm(In - Q.T @ Q)
    m, n_q = Q.shape
    return e1, e2, m, n_q

I tested my Gram Schmidt algorithm by applying it to four test matrices and collecting the error norms into a data frame, which I displayed as a table below.

In [4]:
X1 = np.array([
    [1, 1, 1],
    [0, 1, 1],
    [0, 0, 1]
])
X2 = np.array([
    [1, 1, 1],
    [1, 1, 0],
    [1, 0, 0]
])
X3 = np.array([
    [ 2, -1,  0],
    [-1,  2, -1],
    [0,  -1,  2],
    [ 0,  0, -1]
])
X4 = gallery.hilbert_matrix(10)

data = []
for i, X in enumerate([X1, X2, X3, X4]):
    e1, e2, m, n_q = exercise1(X)
    data.append((f'X{i+1}', e1, e2,m, n_q))

df = pd.DataFrame(data, columns=['matrix', 'e1', 'e2', 'm', 'n_q'])
df

,matrix,e1,e2,m,n_q
0,X1,0.000000e+00,0.000000e+00,3,3
1,X2,6.316753e-16,5.760479e-16,3,3
2,X3,1.000000e+00,2.460790e-16,4,3
3,X4,4.051053e-04,4.051053e-04,10,10


The error norms on the first three matrices are all close to machine epsilon, so the orthogonalization was accurate. However, the errors for the Hilbert matrix are on the order of 4e-4, which is relatively high. We know from previous labs that the Hilbert matrix has a high condition number, which could make it difficult to orthogonalize as well.

Note that the output matrix for $X_3$ is not considered an orthogonal matrix because it is not square, as seen by its different number of rows and columns.

## Exercise 2 - Gram-Schmidt factorization

The goal of this exercise is to modify the Gram Schmidt process to produce a factorization of the matrix $A \in \mathbb{R}^{M \times N}$ into an orthogonal matrix $Q \in \mathbb{R}^{M \times N}$ and an upper triangular matrix $R  \in \mathbb{R}^{N \times N}$ such that:

$$
    A = QR
$$

I created a function `gs_factor(A)` that performs a QR decomposition on the input matrix $A$ using nearly the same algorithm as the Gram Schnidt process. I quickly tested that this produces the same Q matrices as the `gs(X)` function from the previous exercise using the four test matrices.

In [6]:
def gs_factor(A):
    '''
    Factor an M x N matrix A into a N x N
    orthogonal matrix Q and a M x N upper
    triangular matrix R such that A = QR,
    using the Gram Schmidt method.

    Args:
        A: M x N matrix.
    Returns:
        M x N orthogonal matrix Q.
        N x N upper triangular matrix R.
    '''
    m, n = A.shape
    
    # initialize return values
    Q = A.astype(float)
    R = np.zeros((n, n))
    
    # for each column,
    for j in range(n):

        # for each previous column,
        for i in range(j):

            # compute the dot product
            R[i,j] = Q[:,i] @ Q[:,j]
            
            # subtract the projection
            #   this makes them orthogonal
            Q[:,j] -= Q[:,i] * R[i,j]

        # normalize the vector
        R[j,j] = norm(Q[:,j])
        assert R[j,j] > 0, 'zero column'
        Q[:,j] /= R[j,j]

    return Q, R

for i, X in enumerate([X1, X2, X3, X4]):
    Q1 = gs(X)
    Q2 = gs_factor(X)[0]
    print(norm(Q1 - Q2))

0.0
2.220446049250313e-16
0.0
0.00015882294813383433


I created a function `exercise2(A)` that takes a matrix $A$ as input and performs QR factorization using the Gram Schmidt method. Then it computes and returns the error norm of the original matrix compared to the product of the factors.

In [7]:
def exercise2(A):
    Q, R = gs_factor(A)
    e = norm(A - Q @ R)
    return e

For each of the four test matrices from the last exercise, I performed Gram Schmidt factorization and collected the resulting error into a data frame. I displayed the results below.

In [8]:
data = []
for i, X in enumerate([X1, X2, X3, X4]):
    e = exercise2(X)
    data.append((f'X{i+1}', e))

df = pd.DataFrame(data, columns=['matrix', 'error'])
df

,matrix,error
0,X1,0.000000e+00
1,X2,1.158629e-16
2,X3,1.790212e-17
3,X4,1.047750e-16


My Gram Schmidt factorization algorithm achieved a very low error rate on all four test matrices.

## Exercise 3 - Householder matrix

The objective of this exercise is to understand what a Householder matrix is and how we can construct them. A Householder matrix $H$ for a given vector $d$ is defined by the property:

$$
    Hd = \|d\|e_k 
$$

Where $e_k$ is a vector of zeros except for one at index $k$. The matrix can be calculated as:

$$
    H = I - 2vv^\top
$$

Where:
$$
\begin{align}
v = \frac{d - \|d\|e_k}{\|(d - \|d\|e_k)\|}
\end{align}
$$

We can extend this idea to construct a different type of Householder matrix that produces vectors where all entries below $k$ are zero.

I implemented a function `householder(b, k)` below that takes a vector $b$ and an integer $k$ as input and produces a Householder matrix $H$ such that all entries of $Hb$ below index $k$ are zero.

In [9]:
def householder(b, k):
    '''
    Return an orthogonal matrix H
    such that all entries of H b
    below index k are zero.
    
    Args:
        b: Vector of length N.
        k: Scalar index.
    Returns:
        N x N Householder matrix.
    '''
    n, = b.shape
    d = b[k:n]
    alpha = norm(d)
    
    if alpha == 0:
        return np.eye(n)
    
    # choose sign to minimize roundoff errors
    if d[0] > 0:
        alpha = -alpha

    v = np.zeros_like(d, dtype=float)
    v[0] = np.sqrt(1/2*(1 - d[0]/alpha))
    p = -alpha*v[0]
    v[1:] = d[1:]/(2*p)

    # prepend zeros to get w
    w = np.r_[np.zeros(k), v]
    
    # construct householder matrix
    return np.eye(n) - 2*np.outer(w, w)

I created a function `exercise3(b, k)` that tests the above method by computing the Householder matrix for a given vector and index, then computes the error norm $\|I - HH^\top\|$ to check that $H$ is orthogonal. It also returns the transformed vector $c = Hb$.

In [11]:
def exercise3(b, k):
    n, = b.shape
    H = householder(b, k)
    e = norm(np.eye(n) - H.T @ H)
    c =  H @ b
    return e, c

I tested these functions on the vector $b = [10,9,8,7,6,5,4,3,2,1]$ with $k$ values in the range from 0 to 3. I collected the error norms and the norms of the values of $c$ with index greater than $k$, which should be zero.

In [12]:
data = []
b = np.arange(10, 0, -1)
for k in range(4):
    e, c = exercise3(b, k)
    data.append((k, e, norm(c[k+1:])))
    
df = pd.DataFrame(data, columns=['k', 'e', 'c'])
df

,k,e,c
0,0,3.234068e-16,1.227537e-15
1,1,2.618233e-16,2.796021e-15
2,2,4.801947e-16,1.137640e-15
3,3,2.671437e-16,1.355200e-15


As seen in the table above, the error norms are very low, indicating that the generated Householder matrices are orthogonal. In addition, the $c$ column verifies that the expected entries are zero in the transformed vectors.

## Exercise 4 - Sequence of Householder transformations

The intent of this exercise is to observe how sequences of multiplications by Householder matrices can be used to factorize a matrix into $QR$ form. Each Householder transformation sets the values in a column below the diagonal to zero, so finding a set of $N$ Householder matrices that put the matrix into upper triangular form corresponds to finding a $QR$ decomposition.

First, I created a magic matrix $A$ of size $N=5$ and initial values $Q_0=I$ and $R_0=A$.

In [18]:
n = 5
A = gallery.magic_matrix(n)
Q = np.eye(n)
R = np.copy(A)
print_mat(A=A, Q=Q, R=R)

A =
[[17. 24.  1.  8. 15.]
 [23.  5.  7. 14. 16.]
 [ 4.  6. 13. 20. 22.]
 [10. 12. 19. 21.  3.]
 [11. 18. 25.  2.  9.]]
Q =
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
R =
[[17. 24.  1.  8. 15.]
 [23.  5.  7. 14. 16.]
 [ 4.  6. 13. 20. 22.]
 [10. 12. 19. 21.  3.]
 [11. 18. 25.  2.  9.]]


Next, I created a loop over the columns of the matrices. In each iteration $k$, I created a Householder matrix $H_k$ that sets the entries of column $k$ to zero below the diagonal. Once a Householder matrix is created for each column, the result is a set of orthogonal transformations and an upper triangular matrix.

Essentially, the factorization proceeds as follows:

$$
\begin{align}
    A &= I A \\
    A &= I H_1^\top H_1 A \\
    A &= I H_1^\top H_2^\top H_2 H_1 A \\
    A &= I H_1^\top H_2^\top H_3^\top H_3 H_2 H_1 A \\
    A &= I H_1^\top H_2^\top H_3^\top H_4^\top H_4 H_3 H_2 H_1 A \\
    A &= I H_1^\top H_2^\top H_3^\top H_4^\top H_5^\top H_5 H_4 H_3 H_2 H_1 A \\
\end{align}
$$

Which is equivalent to the following QR decomposition:

$$
\begin{align}
    A &= QR \\
    Q &= I H_1^\top H_2^\top H_3^\top H_4^\top H_5^\top \\
    R &= H_5 H_4 H_3 H_2 H_1 A \\
\end{align}
$$

In [19]:
for k in range(n):
    H = householder(R[:,k], k)
    Q = Q @ H.T
    R = H @ R
    e = norm(A - Q @ R)
    print(k, e)
    
print_mat(Q=Q, R=R, QR=Q@R)

0 1.8492463394943048e-14
1 2.4132479426685772e-14
2 2.7620242561485495e-14
3 2.7938823627994634e-14
4 2.7938823627994634e-14
Q =
[[-0.5234  0.5058  0.6735 -0.1215  0.0441]
 [-0.7081 -0.6966 -0.0177  0.0815  0.08  ]
 [-0.1231  0.1367 -0.3558 -0.6307  0.6646]
 [-0.3079  0.1911 -0.4122 -0.4247 -0.72  ]
 [-0.3387  0.4514 -0.4996  0.6328  0.1774]]
R =
[[-32.4808 -26.6311 -21.3973 -23.7063 -25.8615]
 [ -0.      19.8943  12.3234   1.9439   4.0856]
 [  0.       0.     -24.3985 -11.6316  -3.7415]
 [ -0.      -0.       0.     -20.0982  -9.9739]
 [ -0.      -0.      -0.       0.      16.0005]]
QR =
[[17. 24.  1.  8. 15.]
 [23.  5.  7. 14. 16.]
 [ 4.  6. 13. 20. 22.]
 [10. 12. 19. 21.  3.]
 [11. 18. 25.  2.  9.]]


At each iteration of the for loop above, the error output shows that the factorization is accurate. The final $QR$ matrices confirm that the decomposition was successful and the resulting matrices have the expected properties.

## Exercise 5 - Householder factorization

The purpose of this exercise is to implement a new method of computing a QR decomposition by using sequences of Householder transformations, as we previously demonstrated.

I created a function `h_factor(A)` that creates a sequence of Householder matrices that transform the provided matrix into an upper triangular form $R$, with the product of the Householder matrix inverses representing the orthogonal factor $Q$.

In [20]:
def h_factor(A):
    '''
    Factor an M x N matrix A into a M x M
    orthogonal matrix Q and a M x N upper
    triangular matrix R such that A = QR,
    using Householder transformations.

    Args:
        A: M x N matrix.
    Returns:
        M x M orthogonal matrix Q.
        M x N upper triangular matrix R.
    '''
    m, n = A.shape
    
    # initialize return values
    Q = np.eye(m)
    R = A.copy()
    
    # for each column,
    for k in range(n):

        # get Householder matrix that zeros out
        #   the current column below the diagonal
        H = householder(R[:,k], k)
        Q = Q @ H.T
        R = H @ R

    return Q, R

I tested the Householder factorization function on the simple 3 x 3 matrix $A$ below.

In [21]:
A = np.array([
    [0, 1, 1],
    [1, 1, 1],
    [0, 0, 1]
])
Q, R = h_factor(A)
print_mat(A=A, Q=Q, R=R, QR=Q@R)

A =
[[0 1 1]
 [1 1 1]
 [0 0 1]]
Q =
[[-0. -1.  0.]
 [ 1. -0.  0.]
 [ 0.  0. -1.]]
R =
[[ 1.  1.  1.]
 [-0. -1. -1.]
 [ 0.  0. -1.]]
QR =
[[0. 1. 1.]
 [1. 1. 1.]
 [0. 0. 1.]]


The resulting $Q$ factor is a permutation matrix with some negative entries, and the $R$ factor is a rearrangement of the rows of $A$ with some negative rows.

Next, I tested the Householder factorization method on a magic matrix of size 5 and compared the result to the one obtained by Gram Schmidt factorization.

In [27]:
A = gallery.magic_matrix(5)
print_mat(A=A)

print('\nHouseholder factorization')
Q, R = h_factor(A)
print_mat(Q=Q, R=R, QR=Q@R)

print('\nGram-Schmidt factorization')
Q, R = gs_factor(A)
print_mat(Q=Q, R=R, QR=Q@R)

A =
[[17. 24.  1.  8. 15.]
 [23.  5.  7. 14. 16.]
 [ 4.  6. 13. 20. 22.]
 [10. 12. 19. 21.  3.]
 [11. 18. 25.  2.  9.]]

Householder factorization
Q =
[[-0.5234  0.5058  0.6735 -0.1215  0.0441]
 [-0.7081 -0.6966 -0.0177  0.0815  0.08  ]
 [-0.1231  0.1367 -0.3558 -0.6307  0.6646]
 [-0.3079  0.1911 -0.4122 -0.4247 -0.72  ]
 [-0.3387  0.4514 -0.4996  0.6328  0.1774]]
R =
[[-32.4808 -26.6311 -21.3973 -23.7063 -25.8615]
 [ -0.      19.8943  12.3234   1.9439   4.0856]
 [  0.       0.     -24.3985 -11.6316  -3.7415]
 [ -0.      -0.       0.     -20.0982  -9.9739]
 [ -0.      -0.      -0.       0.      16.0005]]
QR =
[[17. 24.  1.  8. 15.]
 [23.  5.  7. 14. 16.]
 [ 4.  6. 13. 20. 22.]
 [10. 12. 19. 21.  3.]
 [11. 18. 25.  2.  9.]]

Gram-Schmidt factorization
Q =
[[ 0.5234  0.5058 -0.6735  0.1215  0.0441]
 [ 0.7081 -0.6966  0.0177 -0.0815  0.08  ]
 [ 0.1231  0.1367  0.3558  0.6307  0.6646]
 [ 0.3079  0.1911  0.4122  0.4247 -0.72  ]
 [ 0.3387  0.4514  0.4996 -0.6328  0.1774]]
R =
[[32.4808 26.63

The two methods produce identical $QR$ decompositions of the matrix, up to differences in sign.

Finally, I tested both the Householder and Gram-Schmidt factorization methods on a Hilbert matrix of size 10, and computed the error in each case.

In [48]:
A = gallery.hilbert_matrix(10)

data = []
for method in [h_factor, gs_factor]:
    Q, R = method(A)
    e_Q = norm(np.eye(10) - Q.T @ Q)
    e_A = norm(A - Q @ R)
    data.append((method.__name__, e_Q, e_A))

pd.DataFrame(data, columns=['method', '$\|I - Q^T Q\|$', '$\|A - QR\|$'])

,method,$\|I - Q^T Q\|$,$\|A - QR\|$
0,h_factor,1.079478e-15,3.126355e-16
1,gs_factor,3.804115e-04,1.047750e-16


The table above shows two different types of error for the QR decomposition algorithms. Both methods produce very low error in terms of the difference between the input matrix $A$ and the factor product $QR$, but the Householder method produces a $Q$ matrix that is much closer to being exactly orthogonal than the Gram Schmidt method.

## Exercise 6 - Solving linear systems with QR decomposition

The goal of this exercise is to see how we can use QR decomposition to solve linear systems. Given a linear system and QR decomposition:

$$
\begin{align}
    Ax &= b \\
    A &= QR \\
    QRx &= b \\
    Rx &= Q^{-1} b \\
    Rx &= Q^\top b
\end{align}
$$

We can then solve this upper triangular system through simple back-substitution.

To verify this, I implemented two functions below `u_solve(U, y)` (that same back-substitution method from the previous lab) and `h_solve(A, b)`. The `h_solve` method performs a $QR$ decomposition on $A$ using `h_factor(A)`, then creates a new righthand side by multiplying by $Q^\top$. The resulting upper triangular system can be solved by a call to `u_solve`.

In [45]:
def u_solve(U, y):
    '''
    Solve Ux = y, where U is
    an upper triangular matrix.
    '''
    # check input shapes
    n = U.shape[0]
    assert U.shape == (n, n)
    assert y.shape == (n,)

    x = y.copy()
    for i in reversed(range(n)):
        x[i] -= U[i,i+1:] @ x[i+1:]
        x[i] /= U[i,i]

    return x

def h_solve(A, b):
    '''
    Solve a linear system Ax = b
    using QR decomposition via
    Householder transformations.
    '''
    # check input shapes
    n, _ = A.shape
    assert A.shape == (n, n)
    assert b.shape == (n,)

    # A = Q R
    Q, R = h_factor(A)
    
    # Rx = Q^T b
    QTb = Q.T @ b
    
    # solve for x
    return u_solve(R, QTb)

I tested the QR solution method by creating a magic matrix $A$ of size 5 and a solution vector $x_1$. Then I computed the righthand side $b = Ax_1$ and applied `h_solve(A, b)` to obtain an approximate solution $x_2$. The error I obtained using this method was 2e-15, indicating that the QR solution method works well.

In [46]:
A = gallery.magic_matrix(5)
x1 = np.arange(5)
b = A @ x1
x2 = h_solve(A, b)
norm(x1 - x2)

2.0873306750654603e-15